This is the notebook for training and predictions of the dispersive (D) HSP component. GNN's are more computationaly intensive, and since we test multiple models, splitting the tasks for each solubility parameter in seperate notebooks can make the outlining code more readable.

The code is the same for hydrogen (H) and polar (P) components.

# Libraries

In [1]:
import deepchem as dc
from rdkit import Chem
import numpy as np
import pandas as pd
import random
import torch

from hyperopt import hp, fmin, tpe, Trials
from deepchem.models import GATModel, GCNModel, AttentiveFPModel, MPNNModel
from deepchem.models.optimizers import Adam

# Set the seed
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)


# Dataset

In [3]:
dataset = pd.read_csv('data/HSP_SMILES.csv',index_col=[0] )
dataset

,al,CAS,smiles,δD,δP,δH
0,"1,1,1,2-Tetrachloroethane",b'630-20-6',ClCC(Cl)(Cl)Cl,18.0,4.4,4.2
1,"1,1,1-Trichloroethane",b'71-55-6',CC(Cl)(Cl)Cl,16.8,4.3,2.0
2,"1,1,1-Trifluoroethane",b'420-46-2',CC(F)(F)F,14.6,10.0,0.0
3,"1,1,2,2-Tetrabromoethane",b'79-27-6',BrC(Br)C(Br)Br,21.0,7.0,8.2
4,"1,1,2,2-Tetrachloroethane",b'79-34-5',ClC(Cl)C(Cl)Cl,18.8,5.1,5.3
...,...,...,...,...,...,...
1195,Quinine,b'130-95-0',[H][C@@]1([C@@H](C2=CC=NC3=CC=C(C=C23)OC)O)C[C...,19.0,6.6,11.0
1196,Sulfur Dioxide,b'9/5/7446',O=S=O,15.8,8.4,10.0
1197,Thionyl Chloride,b'9/7/7719',O=S(Cl)Cl,16.9,6.4,6.1
1198,Triethylene Glycol Monooleyl Ether,b'5274-66-8',COCCOCCOCCO,16.0,3.1,8.4


In [ ]:
dataset_length = dataset.shape[0]

Get the SMILES in a list:

In [ ]:
smiles=dataset['smiles'].to_list()

# Featurizer - SMILES to graphs

Initialize the featurizer and generate molecules from SMILES:

In [ ]:
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True) #featurizer with both node and edge atributes
out = featurizer.featurize(smiles) # molecules from smiles

Each molecule is now represented as a graph, so the task of predicting the solubility parameter is considered as a graph classification task, so we just need to define the targets (the true values of solubility parameters) similarly as for the XGBoost model:

In [ ]:
targets=dataset.iloc[:,-3:].to_numpy() # HSP parameters : D, P, H
targets

## Making the graph dataset

Dispersive parameter is the first column of `targets` (`targets[:,0]`)

In [ ]:
D_dataset = dc.data.NumpyDataset(X=out, y=np.array(targets[:,0].reshape(dataset_length,1))) 

### Splits 

In [ ]:
splitter = dc.splits.RandomSplitter()

D_train_dataset, D_test_dataset = splitter.train_test_split(dataset=D_dataset, frac_train=0.8,seed=0)

# Cross validation splits:
kfold=dc.splits.RandomSplitter()
k=5
D_kfold=kfold.k_fold_split(dataset=D_train_dataset,k=k,seed=seed_value)

# Hyperopt search space

In [ ]:
lr=0.0005

gat_search_space =  {
        'layer_sizes': hp.choice('layer_sizes',[[16], [32], [16,16],[32,32],[64,64], [16,16,16],[32,32,32],[64,64,64]]),
        'dropouts': hp.uniform('dropout',low=0.2, high=0.5)
        }

gcn_search_space = {
        'layer_sizes': hp.choice('layer_sizes',[[16], [32], [16,16],[32,32],[64,64], [16,16,16],[32,32,32],[64,64,64]]),
        'dropouts': hp.uniform('dropout',low=0.2, high=0.5),
        'residual': hp.choice('residual', [True, False])
        }

att_search_space = {
    'dropouts': hp.uniform('dropout',low=0.2, high=0.5),
    'num_layers': hp.choice('num_layers',[1,2,3,4]),
    'graph_feat_size':hp.choice('graph_feat_size',[50,100,200,300]),
    'num_timesteps': hp.choice('num_timesteps',[2,4,6]) }

mpnn_search_space =  {
        'node_out_feats': hp.choice('node_out_feats',[16,32,64]),
        'edge_hidden_feats': hp.choice('edge_hidden_feats',[16,32,64,128]),
        'num_step_message_passing' : hp.choice('num_step_message_passing',[1,2,3,4]),
        'dropouts': hp.uniform('dropout',low=0.2, high=0.5),
        'self_loop':hp.choice('self_loop',[True,False])
        }

In [ ]:
import tempfile #tempfile is used to save the best checkpoint later in the program.

metric = dc.metrics.Metric(dc.metrics.r2_score)
n_epochs = 200
max_evals = 30

In [ ]:
CV_scores = {'GAT':[],'GCN':[],'AttentiveFP':[],'MPNN':[]}
#best_models = {'GAT':0,'GCN':0,'AttentiveFP':0,'MPNN':0}

# Training for model selection (cross validation + hyperopt)

Warning: This can take a while, depending on hardware, but it can take more than 12 hours. You can reduce the number of epochs or `max_evals`, but make sure that the model loss is stabilized or begining to stabilize when you choose the number of epochs. 

In [ ]:

i=1
for train,test in D_kfold:
    print ("Training fold: ",i)

    
    def GAT(args):
        save_dir = tempfile.mkdtemp()
        model = GATModel(n_tasks=1,
                            graph_attention_layers=args['layer_sizes'],
                            dropout=args['dropouts'],
                            optimizer=Adam(learning_rate=lr),device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
            
        #validation callback that saves the best checkpoint, i.e the one with the maximum score.
        validation=dc.models.ValidationCallback(test, 100, [metric],save_dir=save_dir,save_on_minimum=False)
        
        model.fit(train, nb_epoch=n_epochs,callbacks=validation)

        #restoring the best checkpoint and passing the negative of its validation score to be minimized.
        model.restore(model_dir=save_dir)
        valid_score = model.evaluate(test, [metric])

        return -valid_score['r2_score']

    def GCN(args):
        save_dir = tempfile.mkdtemp()
        model = GCNModel(n_tasks=1,graph_convolution_layers=args['layer_sizes'],dropout=args['dropouts'],
                            residual=args['residual'],
                            optimizer=Adam(learning_rate=lr),device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        #validation callback that saves the best checkpoint, i.e the one with the maximum score.
        validation=dc.models.ValidationCallback(test, 100, [metric],save_dir=save_dir,save_on_minimum=False)
        
        model.fit(train, nb_epoch=n_epochs,callbacks=validation)

        #restoring the best checkpoint and passing the negative of its validation score to be minimized.
        model.restore(model_dir=save_dir)
        valid_score = model.evaluate(test, [metric])

        return -valid_score['r2_score']
    
    def AttentiveFP(args):
        save_dir = tempfile.mkdtemp()
        model = AttentiveFPModel(n_tasks=1,num_layers=args['num_layers'],
                                    graph_feat_size=args['graph_feat_size'],
                                    dropout=args['dropouts'], num_timesteps=args['num_timesteps'],
                        optimizer=Adam(learning_rate=lr),device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
            
        #validation callback that saves the best checkpoint, i.e the one with the maximum score.
        validation=dc.models.ValidationCallback(test, 100, [metric],save_dir=save_dir,save_on_minimum=False)
        
        model.fit(train, nb_epoch=n_epochs,callbacks=validation)

        #restoring the best checkpoint and passing the negative of its validation score to be minimized.
        model.restore(model_dir=save_dir)
        valid_score = model.evaluate(test, [metric])

        return -valid_score['r2_score']
    
    def MPNN(args):
        save_dir = tempfile.mkdtemp()
        model = MPNNModel(n_tasks=1,
                            node_out_feats=args['node_out_feats'],
                            edge_hidden_feats=args['edge_hidden_feats'],
                            num_step_message_passing = args['num_step_message_passing'],
                            self_loop=args['self_loop'],                     
                            dropout=args['dropouts'],
                            optimizer=Adam(learning_rate=lr),device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

        
    print("-- GAT TRAINING BEGINS --")
    
    trialsGAT=Trials()
    bestGAT = fmin(GAT,
                space= gat_search_space,
                algo=tpe.suggest,
                max_evals=max_evals,
                trials = trialsGAT, rstate=np.random.default_rng(seed_value))
    
    
    print("-- GCN TRAINING BEGINS --")
    
    trialsGCN=Trials()
    bestGCN = fmin(GCN,
                space= gcn_search_space,
                algo=tpe.suggest,
                max_evals=max_evals,
                trials = trialsGCN, rstate=np.random.default_rng(seed_value))
    
    
    print("-- ATTENTIVEFP TRAINING BEGINS --")
    
    trialsAtt=Trials()
    bestAtt = fmin(AttentiveFP,
                space= att_search_space,
                algo=tpe.suggest,
                max_evals=max_evals,
                trials = trialsAtt, rstate=np.random.default_rng(seed_value))
    
    print("-- MPNN TRAINING BEGINS --")

    trialsMPNN=Trials()
    bestMPNN = fmin(MPNN,
                space= mpnn_search_space,
                algo=tpe.suggest,
                max_evals=max_evals,
                trials = trialsMPNN, rstate=np.random.default_rng(seed_value))
    
    CV_scores['GAT'].append(trialsGAT.trials[0]['result']['loss'])
    CV_scores['GCN'].append(trialsGCN.trials[0]['result']['loss'])
    CV_scores['AttentiveFP'].append(trialsAtt.trials[0]['result']['loss'])
    CV_scores['MPNN'].append(trialsMPNN.trials[0]['result']['loss'])
    
    i+=1

Save the CV results for each model and fold:

In [ ]:
import pickle
out_folder =  "results/gnn/D/"
name = out_folder+f'CV_D_{n_epochs}_{max_evals}.pickle'
# Store data
with open(name, 'wb') as handle:
    pickle.dump(CV_scores, handle, protocol=pickle.HIGHEST_PROTOCOL)


Show the mean performance of models to choose which one to choose for further and final training:

In [ ]:
for key in CV_scores.keys():
    print (f"----- {key} model -----")
    print(f"{key} model mean r2 scores accros folds: {np.mean(CV_scores[key])}")
    print(f"{key} model standard deviation of r2 scores accros folds: {np.std(CV_scores[key])}")

We choose a model with the highest mean score, but if two models have similar performance we choose the one with the smallest standard deviation. For D and H these models are AttentiveFP, while for the P parameter the model that yields slightly better results is GAT.

# Get the best model hyperparameters

Train the model with hyperopt and get the best model and hyperparameters

In [ ]:
metric = dc.metrics.Metric(dc.metrics.r2_score)
n_epochs = 500 
max_evals = 30

Split the data to train and validation and test sets:

In [ ]:
D_train_dataset,D_validation_dataset, D_test_dataset = splitter.train_valid_test_split(dataset=D_dataset, frac_train=0.6,frac_valid=0.2,frac_test=0.2,seed=seed_value)

In [ ]:

def ATT(args):
    save_dir = tempfile.mkdtemp()
    model = AttentiveFPModel(n_tasks=1,num_layers=args['num_layers'],
                             graph_feat_size=args['graph_feat_size'],
                             dropout=args['dropouts'], num_timesteps=args['num_timesteps'],
                optimizer=Adam(learning_rate=args['learning_rate']))
    #validation callback that saves the best checkpoint, i.e the one with the maximum score.
    validation=dc.models.ValidationCallback(D_validation_dataset, 50, [metric],save_dir=save_dir,save_on_minimum=False)

    model.fit(D_train_dataset, nb_epoch=n_epochs,callbacks=validation)

    #restoring the best checkpoint and passing the negative of its validation score to be minimized.
    model.restore(model_dir=save_dir)
    valid_score = model.evaluate(D_validation_dataset, [metric])

    return -valid_score['r2_score']


In [ ]:
trials=Trials()
best = fmin(ATT,
            space= att_search_space,
            algo=tpe.suggest,
            max_evals=max_evals,
            trials = trials, rstate=np.random.default_rng(seed_value))

Get the best hyperparameters:

In [ ]:
# put the .choice arguments from the search space in lists because "best" returns the index of such lists and not the value itself
layers=[1,2,3,4] 
num_layers=[1,2,3,4]
graph_feat_size=[50,100,200,300]
num_timesteps=[2,4,6,8]

best_params = {'dropout':best['dropout'],
               'learning_rate':best['learning_rate'],
               'num_layers':num_layers[best['num_layers']],
               'graph_feat_size':graph_feat_size[best['graph_feat_size']],
               'num_timesteps':num_timesteps[best['num_timesteps']]
              }
print(best_params)

Optionally, save the `best_params` dict

In [ ]:
with open(f'/results/gnn/D/ATT_D_params_{n_epochs}_{max_evals}.pickle', 'wb') as handle:
    pickle.dump(best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Final train and test:

In [ ]:
optimizer = Adam(best_params['learning_rate'])

#initialize and save the model
model = AttentiveFPModel(n_tasks=1,**best_params,optimizer=optimizer,model_dir ='/trained_models/gnn/D/' )
metric1 = dc.metrics.Metric(dc.metrics.r2_score)
metric2 = dc.metrics.Metric(dc.metrics.mean_squared_error)
callback=dc.models.ValidationCallback(D_validation_dataset, 50, [metric1,metric2])
model.fit(D_train_dataset, nb_epoch=n_epochs, callbacks=callback)

Predictions:

In [ ]:
y_all = model.predict(D_dataset)
y_test = model.predict(D_test_dataset)

train_val= dc.data.NumpyDataset.merge([D_train_dataset,D_validation_dataset])
y_train_val = model.predict(train_val)

from sklearn.metrics import r2_score,mean_squared_error
print('ALL r2: ', r2_score(D_dataset.y, y_all))
print('test r2: ', r2_score(D_test_dataset.y, y_test))
print('train-val r2: ', r2_score(train_val.y, y_train_val))

print('ALL MSE: ', mean_squared_error(D_dataset.y, y_all,squared=False))
print('test MSE: ', mean_squared_error(D_test_dataset.y, y_test,squared=False))
print('train-val MSE: ', mean_squared_error(train_val.y, y_train_val,squared=False))

Save the results (optional):

In [ ]:
model_name="AttentiveFP"
TRAIN_PREDS = pd.DataFrame({'real':train_val.y.ravel(),'predicted':y_train_val.ravel()})
TRAIN_PREDS.to_csv(f'results/gnn/D/D_TRAIN_PREDS_{model_name}_{n_epochs}_epochs.csv')

TEST_PREDS = pd.DataFrame({'real':D_test_dataset.y.ravel(),'predicted':y_test.ravel()})
TEST_PREDS.to_csv(f'results/gnn/D/D_TEST_PREDS_{model_name}_{n_epochs}_epochs.csv')